# 2. 전처리(Preprocessing)
- pdf, ppt로부터 불러온 텍스트 데이터를 다듬어주는 과정이다.
- 현재 쓰인 텍스트들은 줄바꿈, 불필요한 특수문자 등이 존재하기 때문에 전처리를 통해 다듬는 과정이 필요하다.

LLM을 사용하여 전처리를 진행하여, 완전한 문장으로 나누어질 수 있게 구분하려 한다.

## 2.1 모델 불러오기 (Ollama3-EEVE)

### 2.1.1 환경 설정
Ollama3 -EEVE 모델을 위한 `.gguf`파일 다운로드

`python magic method` - `%%bash` 를 사용하여 다운받을 수 있다.
- local_directory 부분에 모델을 다운받을 위치를 설정해주자.

```python
%%bash
huggingface-cli download \
  heegyu/EEVE-Korean-Instruct-10.8B-v1.0-GGUF \
  ggml-model-Q5_K_M.gguf \
  --local-dir {local_directory} \
  --local-dir-use-symlinks False
```
<br>
진행 환경에서는 `EEVE-Korean-Instruct-10.8B-v1.0-GGUF` 라는 폴더 생성 후 `.gguf` 파일을 넣어주었다.  
같은 위치에 확장자가 없는 `Modelfile`파일을 생성해주고 아래와 같이 작성해주었다.

#### Modelfile
```
FROM ggml-model-Q5_K_M.gguf

TEMPLATE """{{- if .System }}
<s>{{ .System }}</s>
{{- end }}
<s>Human:
{{ .Prompt }}</s>
<s>Assistant:
"""

SYSTEM """A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions."""

PARAMETER stop <s>
PARAMETER stop </s>
```

In [1]:
"""
%%bash
huggingface-cli download \
  heegyu/EEVE-Korean-Instruct-10.8B-v1.0-GGUF \
  ggml-model-Q5_K_M.gguf \
  --local-dir C:/Users/dm705/Study \
  --local-dir-use-symlinks False
"""

'\n%%bash\nhuggingface-cli download   heegyu/EEVE-Korean-Instruct-10.8B-v1.0-GGUF   ggml-model-Q5_K_M.gguf   --local-dir C:/Users/dm705/Study   --local-dir-use-symlinks False\n'

### 2.1.2 파이썬에서 모델 불러오기(EEVE-Korean-10.8B)

In [2]:
"""
Requirements: pip install langchain
"""

from langchain_community.chat_models import ChatOllama   
from langchain_core.output_parsers import StrOutputParser 
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.callbacks.manager import CallbackManager

llm = ChatOllama(
    model="EEVE-Korean-10.8B:latest",
    temperature=0,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
)

## 2. 프롬프트 템플릿 설정

In [3]:
import torch
if torch.cuda.is_available():
    torch.cuda.set_device(torch.device("cuda:0"))
    torch.cuda.empty_cache()

In [4]:
# 2. 프롬프트 템플릿 만들기
template = """
입력된 텍스트를 전처리하고 문장 단위로 나누어 주세요:
{text}

나뉘어진 문장별로 번호를 매겨 나누어 정리해 주십시오.
결과물 출력 외에 질문이나 답변은 하지 마십시오.
"""
prompt = ChatPromptTemplate.from_template(template)

# 3. 출력 파서 설정하기
output_parser = StrOutputParser()

# 4. LLMChain 만들기
llm_chain = prompt | llm | output_parser

## 3. 전처리 - 문장 구분하기 (using Ollama)

### 3.1. `*.txt` 텍스트 전처리 및 문장 구분하기

- 모델에 넣을 때
  - 모든 줄을 한 번에 읽으니, 출력 길이가 길지 않아서 막히는 것 같다.
  - 20줄씩 읽어서 텍스트를 전처리해주려고 한다.

In [5]:
import re

def read_file_in_chunks(file_path, chunk_size=10):
    with open(file_path, 'r', encoding='utf-8') as file:
        while True:
            lines = []
            for _ in range(chunk_size):
                line = file.readline()
                if not line:
                    break
                lines.append(line.strip())
            if not lines:
                break
            yield lines

def cleansing(text_list):
    cleaned_sentence_list = []
    for sentences in text_list:
        sub_sentences = re.sub(r'\r', '', sentences) # 정규표현식으로 '\r'을 제거
        sub_sentence_split = sub_sentences.split('\n') # 모든 문장을 '\n'으로 스플릿
        sub_sentence_sequence = list(map(lambda x:x[3:].strip(), sub_sentence_split)) # 문장마다 붙은 번호를 지움
        cleaned_sentence = '\n\n'.join(sub_sentence_sequence) # 다시 문장을 이어붙여줌. 구분자는 '\n\n'
        cleaned_sentence_list.append(cleaned_sentence)
        
    # for문이 끝나면 나눠진 문장들을 다시 하나로 통합
    cleaned_text = '\n\n'.join(cleaned_sentence_list)
    return cleaned_text

In [14]:
import os
import re
directory = './dataset/'
files = os.listdir(directory)

# 모든 파일에 대해 전처리 실행
txt_files = [file for file in files if file.endswith('.txt')]

for file_name in txt_files:
    file_path = directory + file_name
    
    text_list = []
    
    for chunk in read_file_in_chunks(file_path, chunk_size=20):
        text_list.append(llm_chain.invoke({"text":chunk}))
    
    cleaned_text = cleansing(text_list)

    with open(f'{directory}{file_name[:-4]}_preprocessed.txt', 'w', encoding='utf-8') as file:
        file.write(cleaned_text)

1. 배포일: 4월8일/ (총8매)
2. 담당부서: 보건복지부장애인정책과
3. 과장: 정충현
4. 전화번호: 02-2023-8190
5. 담당자: 이진희, 02-2023-8193
6. 제 33회 장애인의 날 교육자료
7. <장애인의 삶과 사회적 인식> 학습지도안
8. 기획의도: 제 33회 장애인의 날을 맞이하여 학생들이 장애인 삶을 이해하고 편견 없는 시선으로 바라보도록 지도할 수 있는 교육자료 제작 및 배포를 통해, 학생들이 장애인과 비장애인이 더불어 행복한 사회를 만드는 데 기여하도록 함.1. 드는 데 스스로의 역할이 중요함을 자각하고, 배려를 실천할 수 있도록 독려합니다.
2. 활용: 장애인 주간(4월 20일 ~26일)에 교육 권장.
3. 주의: 교재 내 동영상 시청을 위하여 인터넷 연결 및 플래시 플레이어 최신 버전 업데이트가 되어있는지 확인 부탁드립니다.
4. 학습지도안: 주제 - 장애인 삶과 사회적 인식.
5. 지도의 주안점: 평소 학생들이 장애인에 대해 갖고 있는 막연한 두려움이나 무관심 등을 자유롭게 논의할 수 있도록 유도합니다.
6. 질문: 길이나 학교에서 장애인 만나면 어떤 생각이 드나요? 장애인 만나면 어떻게 대하나요?
7. 같은 학급(학교)에 장애인 학생이 있는지? 친구를 놀릴 때 장애인 비하하는 표현을 쓰고 있지 않은지?
8. 2 - 장애인이란?
9. 지도의 주안점: 흔히 알려진 장애유형 외에도 다양한 원인으로 불편을 겪는 사람들이 있다는 사실을 알리고, 우리 모두가 각자 개성 있게 조금씩 다르게 태어난 것과 같이 장애인도 존중받아야 하는 존재임을 이해할 수 있도록 소개합니다.1. 장애인: 신체 일부나 정신적으로 장애가 있어서 일상생활이나 사회생활에 제약이 있는 사람입니다.
2. 장애인 분류: 다양한 종류의 장애가 존재합니다. 바깥으로 드러난 장애 종류만 알고 있지만, 내부 기관의 문제로 인해 장애를 겪는 사람들도 많습니다.
3. 우리 모두가 각자 개성 있게 조금씩 다르게 태어난 것과 같이, 장애인도 존중받아야 할 존재입니다.
4. <참

In [6]:
import os
import re
directory = './dataset/'
files = os.listdir(directory)

# 모든 파일에 대해 전처리 실행
# txt_files = [file for file in files if file.endswith('.txt')]
txt_files = ['data9.txt']

for file_name in txt_files:
    file_path = directory + file_name
    
    text_list = []
    
    for chunk in read_file_in_chunks(file_path, chunk_size=10):
        text_list.append(llm_chain.invoke({"text":chunk}))
    
    cleaned_text = cleansing(text_list)

    with open(f'{directory}{file_name[:-4]}_preprocessed_chunk10.txt', 'w', encoding='utf-8') as file:
        file.write(cleaned_text)

1. 이 교재는 국가인권위원회 사이버인권교육과정 <장애인 차별 예방>의 학습내용을 정리한 것입니다.
2. 저작권은 사이버인권교육과정 원고 집필자인 김성연 (장애인차별 금지추진연대) 저자와 공유하고 있으므로, 상업용으로 사용할 수 없습니다.
3. <장애인 차별 예방> 사이버인권교육을 수강할 때 보조교재로 활용하시기 바랍니다.
4. 사이버 인권교육과정은 국가인권위원회 홈페이지 인권교육센터에서 회원가입 후 로그인하면 무료로 수강하실 수 있습니다.
5. *국가인권위원회 인권교육센터 홈페이지는 http://edu.humanrights.go.kr-i 입니다.
6. 제1강 차이·차별 이야기:
7. 1. 차이와 차별의 개념:
8. 2. 차이1. 차별의 상황들 ·······························1. 제3강 발달, 지적, 정신장애인을 만나다 ·······························1. 제4강 지체·뇌병변장애인을 만나다 ·······························1. 시각, 청각장애의 개념 ·······························1. 장애인 차별금지법 개념
2. 장애인 차별금지법의 의의
3. 장애인 차별금지법 제정 목적과 구성
4. 장애인 차별금지법에 담기지 못한 이야기
5. 공유하기
6. 문제풀기
7. 정리하기
8. 제7강: 차별의 판단 및 정당한 편의 제공
9. 장애인 차별금지법에서의 장애인 차별
10. 장애인 차별금지법 명시된 장애인 차별 상황들1. 3. 공유하기 ·······························1. 일터와 학교에서의 장애인 차별 상황들 ·······························1. 제11강 장애를 이유로 하는 괴롭힘들 ·······························문장을 구분하고 번호를 붙인 결과물은 다음과 같습니다:

1. 문제풀기 ·······························1. 권리옹호를 위한 방법과 제도를 활용한 상황들 ·············

In [7]:
import os
import re
directory = './dataset/'
files = os.listdir(directory)

# 모든 파일에 대해 전처리 실행
# txt_files = [file for file in files if file.endswith('.txt')]
txt_files = ['data12.txt']

for file_name in txt_files:
    file_path = directory + file_name
    
    text_list = []
    
    for chunk in read_file_in_chunks(file_path, chunk_size=5):
        text_list.append(llm_chain.invoke({"text":chunk}))
    
    cleaned_text = cleansing(text_list)

    with open(f'{directory}{file_name[:-4]}_preprocessed_chunk5.txt', 'w', encoding='utf-8') as file:
        file.write(cleaned_text)

1. 편견 을 걷어내면  인재 가 보입니다 직장 내 장애인 인식개선 교육 장애인 고용
2. 장애인 고용 ,  기업에게 어떤 의미가 있을까 함께 일하는 동료 이해하기  장애인 고용 ,  이렇게 할 수 있습니다  함께 일하기 ,  이렇게 하면 잘 할 수 있습니다  01 02 03 04 목  차
3. 01 장애인 고용 , 기업에게 어떤 의미가 있을까
4. 차별도 우대도 없었던   게임 산업 분위기 속  장애인 고용의 돌파구 2011 년 ㈜ 넥슨커뮤니케이션즈 ( 넥슨컴즈 ) 를 설립하고   넥슨의 게임 운영 서비스지원과 고객 지원 업무를 담당함 * 게임 홈페이지 관리 ,  게임 상담 ,  모니터링 등 4 개월간의 직무훈련을 거친 장애인  23 명이   처음으로 산업에 입사하게 됨 실력으로 당당히  No.1  게임 기업의 일원이 되다 ! 장애인고용 기업 사례 넥슨컴즈의  장애인 고용 주력부서  웹서비스 운영팀 상시 근로자  2,691 명  /  장애인 근로자  33 명 (2.04%) * 2018.12 월 기준 모든 출입문 문턱 제거 통로 간격 확장 핸드레일 78% 가 장애인 직원 이중  중증장애인 비율  73%
5. 장애인 업무의 한계를 없애다   상시  근로자  100,016 명  /  장애인  근로자  1,386 명 (1.54%)  * 2018.12 월 기준 장애물 없는 생활환경  (Barrier Free) 자체 인증 제도를 각 사업장1. 기업 ESG 경영 전략으로의 '장애인 고용' 환 경 사 회 지배구조 | ESG 경영전략 | 사회적 가치실현 | 기업이 사회적 책임 완수할 수 있는 가장 효과적인 방법의 일환으로 '장애인 고용'에 대한 관심 확대
2. 기업의 사회적 책임 이행 지속 가능경영 수단 장애인 노동력 활용 미고용 부담금 감면 고용장려금 수혜
3. 기업의 장애인 고용 이점 - 장애인고용의 의미
4. 다양성 관리 전략으로의 '장애인 고용' 장애인고용의 의미 | 다양성 관리 | 다양한 근로자들의 적절한 능력 배합 조직 이익 극대화 포춘지 선정 100 대 기업의 